## DeepLabCut Workflow Notebook

#### This notebook will be divided into two different parts. 
* The first section will handle annotation and training your model within DLC
* The second will handle running inference on unseen video within a pretrained model

#### Login to the database and load your modules:

In [ ]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
dj.config.load('dj_local_config.json')
dj.conn()

from workflow.pipeline import model, train, reference, subject, session
from workflow.pipeline.dlc import insert_new_dlc_model, ingest_behavior_videos
from pathlib import Path
import datetime
import pandas as pd
import yaml

### The tensorflow version installed with the DataJoint DLC elements package is *not* GPU compatible. If you'd like to use a GPU you will need to install a compatible version of tensorflow_gpu. 

In [ ]:
import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## 1. Train workflow

#### Start with annotating video in DLC using napari. 
**Please note that to train your workflow using the supported GUI, you will need to install DLC within the sabatini-datajoint environment separately. Use the following command `pip install napari-deeplabcut'` . This will install the supported GUI and allow you to annotate video within this notebook

#### The cell below is the main manual entry cell

In [ ]:
# Manual entry --
desiredProjectName = 'TW_SideCamera' #Project name for DLC prefix
desiredUserName = 'TW' #Scorer name that will be used in config.yaml
modelDescription = 'Tom_reaching_sideCamera' # Description of the model
paramset_desc = desiredProjectName
paramset_idx = 4
video_set_id = 1
training_id = 3
bodyParts = ['leftIndex','rightIndex','tongue']
skeletonParts = ['leftIndex','tongue']
trainingFraction = [0.8]


rootPath = r'D:/Janet_DJ_test'
lutPath = r'D:/Janet_DJ_test/dlcLUT20231026.xlsx'

#### Here, we will run on headless DLC, then import into the database in the following section

In [ ]:
import deeplabcut 

#### Create your project directory and select videos that you will use

In [ ]:
#Import the lookup table for all the videos you will be using for training
dataFrame = pd.read_excel(lutPath)

In [ ]:
# Automatically determine desired working directory and video files within it
inboxPath = os.path.join(rootPath,'Inbox')
desiredWorkingDirectory = os.path.join(inboxPath,'dlc_projects')
videoPathList = dataFrame.VIDEOPATH_INBOX.tolist()

In [ ]:
# Automatically create a new DLC project
deeplabcut.create_new_project(desiredProjectName, desiredUserName, videoPathList, 
                              working_directory=desiredWorkingDirectory, copy_videos=True, multianimal=False)

In [ ]:
# Automatically determine project path
allProjectNames = os.listdir(desiredWorkingDirectory)
projectNameBase = desiredProjectName+'-'+desiredUserName
projectFileName = next((currProjectName for currProjectName in allProjectNames if projectNameBase in currProjectName), None)
projectPathName = os.path.join(desiredWorkingDirectory,projectFileName)
config_path = os.path.join(projectPathName,r'config.yaml')

In [ ]:
#add additional videos as needed
#deeplabcut.add_new_videos(config_path, ['full path of video 4', 'full path of video 5'], copy_videos=True/False)

### Update your config file

In [ ]:
## Update the config file
with open(config_path, 'rb') as y:
    config_params = yaml.safe_load(y)
    
# Add some variables that aren't originally in the config_parans
training_params = {'shuffle': '1',
                   'trainingsetindex': '0',
                   'maxiters': '100000',
                   'scorer_legacy': 'False',
                   'multianimalproject':'False'}

config_params.update(training_params)


## NB: THE SKELETON INPUT ISN'T QUITE RIGHT BELOW, BUT YOU CAN FIX IT MANUALLY LATER
# Also set some params default to you and params from TVK.
paramsToChange = {'bodyparts':bodyParts,
                 'skeleton':skeletonParts,
                 'TrainingFraction':trainingFraction}

config_params.update(paramsToChange)

In [ ]:
with open(config_path, 'w') as yaml_file:
    yaml_file.write(yaml.dump(config_params, default_flow_style=False))

### Run some DLC functions and upload your model to data joint

#### First, extract frames that will be saved into .pngs for labeling

In [ ]:
deeplabcut.extract_frames(config_path, mode='automatic', algo='kmeans', userfeedback=False, crop=True)

#### Then, label the .pngs with your bodypart points:

In [ ]:
deeplabcut.label_frames(config_path)

In [ ]:
deeplabcut.check_labels(config_path, visualizeindividuals=True)

#### Create a training dataset, remember the train/test split is set in your config.yaml

In [ ]:
deeplabcut.create_training_dataset(config_path, net_type = 'resnet_50', augmenter_type='imgaug')

#### Then, you can upload your model information into DataJoint and train your model which will be ingested automatically

In [ ]:
train.TrainingParamSet.insert_new_params(paramset_idx=paramset_idx, paramset_desc=paramset_desc, params=config_params)

### 1.1) Inserting list into train.Videoset(). Either using LUT or manually

#### 1.1A) Insert list into train.Videoset() using LUT

In [ ]:
labeledDataFolderName = os.path.join(projectPathName,'labeled-data')
videoFolderName = os.path.join(projectPathName,'videos')

allLabeledDataFolders = os.listdir(labeledDataFolderName)
allLabeledDataFolders = [currString for currString in allLabeledDataFolders if not("labeled") in currString]
allLabeledDataPaths = [os.path.join(labeledDataFolderName,currString) for currString in allLabeledDataFolders]
allLabeledDataPathsClipped = [os.path.join('labeled-data',currString) for currString in allLabeledDataFolders]

labelledDataFiles = []
for idx in range(len(allLabeledDataPaths)):
    currFolderPath = allLabeledDataPaths[idx]
    currClippedFolderPath = allLabeledDataPathsClipped[idx]
    for currFilePath in os.listdir(currFolderPath):
        labelledDataFiles.append(os.path.join(currClippedFolderPath,currFilePath))

allVideoFiles = os.listdir(videoFolderName)
allVideoFiles = [os.path.join('videos',currString) for currString in allVideoFiles]

training_filesPreFormatting = labelledDataFiles + allVideoFiles
training_files = ['\\'+s for s in training_filesPreFormatting]

In [ ]:
#project_folder = os.path.join(' ','dlc_projects',projectFileName)[1:]
project_folder = 'dlc_projects\\' +projectFileName


train.VideoSet.insert1({'video_set_id': video_set_id})
for idx, filename in enumerate(training_files):
    currFileName = project_folder + filename
    train.VideoSet.File.insert1({'video_set_id': video_set_id,'file_id': idx,'file_path':currFileName})

alteredProjectPathName = projectPathName.replace('/','\\')

key={'video_set_id': video_set_id,
     'paramset_idx':paramset_idx,
     'training_id': training_id,
     'project_path':alteredProjectPathName
     }
train.TrainingTask.insert1(key, skip_duplicates=True)

#### 1.1B) Insert list into train.Videoset() manually

In [ ]:
# video_set_id = 22
# project_folder = r'Subject4/Session1/dlc_behavior_videos/Tom_test-Tom-2023-10-25'
# training_files = ['/labeled-data/camera1Video2023-10-09T17_21_44/CollectedData_Tom.h5',
#                   '/labeled-data/camera1Video2023-10-09T17_21_44/CollectedData_Tom.csv',
#                   '/labeled-data/camera1Video2023-10-09T17_21_44/img000741.png',
#                   '/labeled-data/camera1Video2023-10-09T17_21_44/img012236.png',
#                   r'/videos/camera1Video2023-10-09T17_21_44.avi']


# # project_folder = r'Subject4\Session1\dlc_behavior_videos\Tom_test-Tom-2023-10-25'
# # training_files = ['\labeled-data\camera1Video2023-10-09T17_21_44\CollectedData_Tom.h5',
# #                   '\labeled-data\camera1Video2023-10-09T17_21_44\CollectedData_Tom.csv',
# #                   '\labeled-data\camera1Video2023-10-09T17_21_44\img000741.png',
# #                   '\labeled-data\camera1Video2023-10-09T17_21_44\img012236.png',
# #                   r'\videos\camera1Video2023-10-09T17_21_44.avi']


# train.VideoSet.insert1({'video_set_id': video_set_id})
# for idx, filename in enumerate(training_files):
#     train.VideoSet.File.insert1({'video_set_id': video_set_id,
#                                  'file_id': idx,
#                                  'file_path': (project_folder + filename)})

#for project path, it should be the full path with your slashes '/'
# key={'video_set_id': video_set_id,
#      'paramset_idx':paramset_idx,
#      'training_id': 11,
#      'project_path':'D:/Janet_DJ_test/Inbox/Subject4/Session1/dlc_behavior_videos/Tom_test-Tom-2023-10-25'
#      }
# train.TrainingTask.insert1(key, skip_duplicates=True)

### 1.2) Run training and log model in database

In [ ]:
train.ModelTraining.populate(key)

In [ ]:
model.BodyPart.extract_new_body_parts(config_path)

In [ ]:
#insert the model into the model.Model schema
model.Model.insert_new_model( model_name=projectFileName, dlc_config=config_path,
                             shuffle=1,trainingsetindex=training_id,
                             model_description=modelDescription,
                             paramset_idx=paramset_idx,
                             params={"snapshotindex":-1})

### 1.3) Evaluate model

In [ ]:
model_query = (model.Model & 'model_name LIKE "%' + projectFileName +'%"')
model.ModelEvaluation.populate(model_query)
model.ModelEvaluation()

#### Now, your model is trained and you can run inference in step 2 below

## 2. Running inference on video(s) with a pretrained model with recusive looping. 

#### You will need a look-up table (LUT). You can find an example within the notebooks directory.

In [ ]:
lutPath = r'D:/Janet_DJ_test/dlcLUT20231026.xlsx'
#Import the lookup table for all the videos you will be running inference on
dataFrame = pd.read_excel(lutPath)

#### Login to the database if you haven't done so above

In [ ]:
import os
if os.path.basename(os.getcwd()) == "notebooks": os.chdir("..")
import datajoint as dj
dj.config.load('dj_local_config.json')
dj.conn()

from workflow.pipeline import model, train, reference, subject, session
from workflow.pipeline.dlc import insert_new_dlc_model, ingest_behavior_videos
from pathlib import Path
import datetime

### Insert model then run inference

In [ ]:
# Display model.Model() table to see insert
model.Model()

In [ ]:
# Display model.BodyPart()
# all body parts across all models
model.BodyPart()

In [ ]:
# Display model.Model.BodyPart()
# body parts associated with this model
model.Model.BodyPart()

### 2.1) Insert device(s). Either using LUT or manually.

#### 2.1A) Insert device(s) using LUT

In [ ]:
allDeviceKeys = reference.Device.fetch(as_dict=True)
allDevicePrimaryKeyData = reference.Device.fetch("KEY")
allDevicePrimaryKeys = list(allDevicePrimaryKeyData[0].keys())

uniqueDeviceData = dataFrame.drop_duplicates(subset=["DEVICE_ID"]) 
for idx in uniqueDeviceData.index:
    deviceId = uniqueDeviceData['DEVICE_ID'][idx]
    deviceName = uniqueDeviceData['DEVICE_NAME'][idx]
    deviceDescription = uniqueDeviceData['DEVICE_DESCRIPTION'][idx]

    reference_dict = dict(device_id=deviceId, device_name=deviceName, device_description=deviceDescription)

    reference_dictPrimaryOnly = dict([((currKey,reference_dict[currKey])) for currKey in allDevicePrimaryKeys])

    ## Check if the dictionary you are planning to log into the database exactly exists already.
    if any([currKey == reference_dict for currKey in allDeviceKeys]):
        print(deviceName + ' record exact already')
        continue

    ## If not, then check if your primary key data already exists in the database already.
    if any([currKey == reference_dictPrimaryOnly for currKey in allDevicePrimaryKeyData]):
        print(str(deviceId) + ' id already in use. You need to update your LUT and reload it in this notebook')
        continue
        
    # Else, insert your dictionary into the database.
    reference.Device.insert1(reference_dict)

#### 2.1B) Insert device(s) manually

In [ ]:
# # Manual step to insert Camera(s) into reference.Device() table
# # This step is required for the model.VideoRecording table to be populated
# deviceId = 0
# deviceName = 'SideCameraTW' 
# deviceDescription = 'Side Camera Tom'
# reference_dict = dict(
#     device_id=deviceId,
#     device_name=deviceName, 
#     device_description=deviceDescription
#     )
# reference.Device.insert1(reference_dict)

### 2.2) Insert subject(s). Either using LUT or manually.

#### 2.2A) Insert subject(s) using LUT

In [ ]:
allSubjectKeys = subject.Subject.fetch(as_dict=True)
allSubjectPrimaryKeyData = subject.Subject.fetch("KEY")
allSubjectPrimaryKeys = list(allSubjectPrimaryKeyData[0].keys())

allDicts = subject.Subject.fetch(as_dict=True)
# uniqueSubjectData = dataFrame.drop_duplicates(subset=["ANIMAL", "NICKNAME","SEX","BIRTHDATE","SUBJECTDESCRIPTION"], keep=False) 
uniqueSubjectData = dataFrame.drop_duplicates(subset=['SUBJECT_NAME']) 
for idx in uniqueSubjectData.index:
    
    subjectName = ('TW_' + str(uniqueSubjectData['SUBJECT_NAME'][idx]))
    subjectNickname = (uniqueSubjectData['SUBJECT_NICKNAME'][idx])
    subjectSex = uniqueSubjectData['SUBJECT_SEX'][idx]
    subjectBirthDate = pd.to_datetime(uniqueSubjectData['SUBJECT_BIRTHDATE'][idx], format='%Y%m%d')
    subjectDescription = 'D' #uniqueSubjectData['SUBJECT_DESCRIPTION'][idx
    
    subj_key = dict(
        subject=subjectName,
        subject_nickname=subjectNickname,
        sex=subjectSex,
        subject_birth_date=subjectBirthDate,
        subject_description= subjectDescription
    )

    subj_keyPrimaryOnly = dict([((currKey,subj_key[currKey])) for currKey in allSubjectPrimaryKeys])

    if any([currKey == subj_key for currKey in allSubjectKeys]):
        print(subjectName + ' record exact already')
        continue

    ## If your subject hasn't already been added, but the name is in use, throw warning and don't try to add
    if any([currKey == subj_keyPrimaryOnly for currKey in allSubjectPrimaryKeyData]): #any([currRow['subject'] == subjectName for currRow in allSubjectPrimaryKeyData]):
        print(subjectName + ' name already in use')
        continue

    ## If the subject name isn't already in use, add it
    subject.Subject.insert1(subj_key)

#### 2.2B) Insert subject(s) manually

In [ ]:
# # Manual step to insert behavior videos into model.VideoRecording() table
# # and into its respective part table model.VideoRecording.File()
# # Specify subject and session_id of the behavior videos
# subjectName = 'TW_102'
# subjectNickname = "WT"
# subjectSex = 'M'
# subjectBirthDate = datetime.date(2023, 4, 13)
# subjectDescription = "Tom Reaching"

# # Make sure specified subject exists within the subject.Subject() table  
# # Skip this step if subject has been inserted into subject.Subject already
# subj_key = dict(
#     subject=subjectName,
#     subject_nickname=subjectNickname,
#     sex=subjectSex,
#     subject_birth_date=subjectBirthDate,
#     subject_description= subjectDescription
# )
# subject.Subject.insert1(subj_key)

### 2.3) Insert session(s). Either using LUT or manually.

#### 2.3A) Insert session(s) using LUT

In [ ]:
allSessionKeys = session.Session.fetch(as_dict=True)
allSessionPrimaryKeyData = session.Session.fetch("KEY")
allSessionPrimaryKeys = list(allSessionPrimaryKeyData[0].keys())

allSessionDirectoryKeys = session.SessionDirectory.fetch(as_dict=True)
allSessionDirectoryPrimaryKeyData = session.SessionDirectory.fetch("KEY")
allSessionDirectoryPrimaryKeys = list(allSessionDirectoryPrimaryKeyData[0].keys())

uniqueSessionData = dataFrame.drop_duplicates(subset=["SUBJECT_NAME","SESSION"]) 
for idx in uniqueSessionData.index:
    # print(str(idx))
    subjectName = ('TW_' + str(uniqueSessionData['SUBJECT_NAME'][idx]))
    sessionId = (uniqueSessionData['SESSION'][idx])
    sessionDateTime = pd.to_datetime(uniqueSessionData['SESSION'][idx], format='%Y%m%d')
    session_key = dict(subject=subjectName, session_id=sessionId,session_datetime=sessionDateTime)
    session_keyPrimaryOnly = dict([((currKey,session_key[currKey])) for currKey in allSessionPrimaryKeys])
    
    if any([currKey == session_key for currKey in allSessionKeys]):
        print(subjectName  + ' ' + str(sessionId) + ' record exact already in sessionKeys')
    elif any([currKey == session_keyPrimaryOnly for currKey in allSessionPrimaryKeyData]): #(any([currRow['subject'] == subjectName for currRow in allSessionKeys]) & any([currKey['session_id'] == sessionId for currKey in allSessionKeys])):
        ## Skip if your subjectxsession has been added before
        print(subjectName + ' name already in use in sessionKeys')
    else:
        session.Session.insert1(session_key)

    session_dir = os.path.join(subjectName,str(sessionId))
    sdir_key = dict(subject=session_key['subject'], session_id=session_key['session_id'],  session_dir=session_dir)
    sdir_keyPrimaryOnly = dict([((currKey,sdir_key[currKey])) for currKey in allSessionDirectoryPrimaryKeys])

    if any([currKey == sdir_keyPrimaryOnly for currKey in allSessionDirectoryKeys]):
        print(subjectName  + ' ' + str(sessionId) + ' record exact already in sessionDirectoryKeys')
    elif any([currKey == sdir_keyPrimaryOnly for currKey in allSessionDirectoryPrimaryKeyData]): #(any([currRow['subject'] == subjectName for currRow in allSessionDirectoryKeys]) & any([currKey['session_id'] == sessionId for currKey in allSessionDirectoryKeys])):
        ## Skip if your subjectxsession has been added before
        print(subjectName + ' name already in use in sessionDirectoryKeys')
    else: 
        session.SessionDirectory.insert1(sdir_key)

#### 2.3B) Insert session(s) manually

In [ ]:
# sessionId = 20231008
# sessionDateTime = '2023-10-08 12:00:00'
# session_key = dict(subject=subjectName, session_id=sessionId,
#                    session_datetime=sessionDateTime)
# session_dir = os.path.join(subjectName,str(sessionId))
# # Make sure specified session exists in session.Session() and session.SessionDirectory() tables
# # Skip these steps if session has been inserted into session tables already
# session.Session.insert1(session_key)
# sdir_key = dict(subject=session_key['subject'], 
#                 session_id=session_key['session_id'], 
#                 session_dir=session_dir)
# session.SessionDirectory.insert1(sdir_key)

### 2.4) Insert video(s). Either using LUT or manually.

#### 2.4A) Insert video(s) using LUT

In [ ]:
allRecordingKeys = model.VideoRecording.fetch(as_dict=True)
allRecordingPrimaryKeyData = model.VideoRecording.fetch("KEY")
allRecordingPrimaryKeys = list(allRecordingPrimaryKeyData[0].keys())

allRecordingFileKeys = model.VideoRecording.File.fetch(as_dict=True)
allRecordingFilePrimaryKeyData = model.VideoRecording.File.fetch("KEY")
allRecordingFilePrimaryKeys = list(allRecordingFilePrimaryKeyData[0].keys())

# allRecordingKeys = model.VideoRecording.fetch(as_dict=True)
# allRecordingFileKeys = model.VideoRecording.File.fetch(as_dict=True)

uniqueRecordingData = dataFrame.drop_duplicates(subset=["SUBJECT_NAME","SESSION","RECORDING"]) 
for idx in uniqueRecordingData.index:
    subjectName = ('TW_' + str(uniqueRecordingData['SUBJECT_NAME'][idx]))
    sessionId = str(uniqueRecordingData['SESSION'][idx])
    recordingId = str(uniqueRecordingData['RECORDING'][idx])
    device_id = (uniqueRecordingData['DEVICE_ID'][idx])

    behavior_key = dict(subject=subjectName,session_id=sessionId)   
    ingest_behavior_videos(behavior_key, device_id, recording_id=recordingId)

    recording_key = {'subject': subjectName,
       'session_id': str(sessionId),
       'recording_id': str(recordingId),
       'device_id':device_id}
    recording_keyPrimaryOnly = dict([((currKey,recording_key[currKey])) for currKey in allRecordingPrimaryKeys])

    
    if any([currKey == recording_keyPrimaryOnly for currKey in allRecordingPrimaryKeyData]):
        print(subjectName + '_' + str(sessionId) + '_' + ': primary key values already in use in model.VideoRecording')
    else:
        model.VideoRecording.insert1({**recording_key},skip_duplicates = True)

    _ = recording_key.pop('device_id')

    f = Path(uniqueRecordingData['VIDEOPATH_INBOX'][idx])
    fileId = uniqueRecordingData['FILE'][idx]
    recording_keyDict = { **recording_key, 'file_id': fileId, 'file_path': f}
    recording_keyList = [recording_keyDict]
    # tmpPath = [{**recording_key, 'file_id': v_idx, 'file_path': Path(f)} for v_idx, f in enumerate(video_files)]
    recording_keyPrimaryOnly = dict([((currKey,recording_keyDict[currKey])) for currKey in allRecordingFilePrimaryKeys])
    
    # print(recording_key.keys())
    if any([currKey == recording_keyPrimaryOnly for currKey in allRecordingFilePrimaryKeyData]):
        print(subjectName + '_' + str(sessionId) + '_' + ': primary key values already in use in model.VideoRecording.File')
    else:
        model.VideoRecording.File.insert(recording_keyList,skip_duplicates = True)

    rec_info_key = (model.RecordingInfo().key_source & 'subject="' + subjectName +'"' & 'session_id="' + sessionId +'"' & 'recording_id="' + recordingId +'"').fetch1("KEY")
    model.RecordingInfo.populate(rec_info_key)

#### 2.4B) Insert video(s) manually

In [ ]:
# recordingId = 1;
# behavior_key = dict(
#     subject=subjectName,
#     session_id=sessionId
#     )   

# # Ingest behavior videos for specified behavior key and device_id 
# # recording_id is assumed to be 0 (1 recording per session)
# ingest_behavior_videos(
#     behavior_key, 
#     device_id,
#     recording_id=recordingId
# )

# # Display model.VideoRecording table
# recording_key = {'subject': subjectName,
#        'session_id': str(sessionId),
#        'recording_id': str(recordingId),
#         'device_id':device_id}
# model.VideoRecording.insert1({**recording_key}, skip_duplicates=True)

# # Display model.VideoRecording.File table
# video_files = [r'D:\Janet_DJ_test\Inbox\TW_102\20231008\camera3Video2023-10-08T19_22_02.avi']

# _ =recording_key.pop('device_id')

# tmpPath = [{**recording_key, 'file_id': v_idx, 'file_path': Path(f)} for v_idx, f in enumerate(video_files)]
# model.VideoRecording.File.insert(tmpPath)

# Populate automated model.RecordingInfo table containing file metadata
# This step can be run after model.VideoRecording() and model.VideoRecording.File() tables are populated
# rec_info_key = (model.RecordingInfo.key_source & 'subject="' + subjectName +'"').fetch1("KEY")
# model.RecordingInfo.populate(rec_info_key)

### 2.5) Insert pose estimation task (inference task)(s). Either using LUT or manually.

#### 2.5A) Insert pose estimation task (inference task)(s) using LUT

In [ ]:
# allPoseEstimationKeys = model.PoseEstimationTask.fetch(as_dict=True)
# allPoseEstimationPrimaryKeyData =model.PoseEstimationTask.fetch("KEY")
# allPoseEstimationPrimaryKeys = list(allPoseEstimationPrimaryKeyData[0].keys())

uniqueRecordingData = dataFrame.drop_duplicates(subset=["SUBJECT_NAME","SESSION","RECORDING"]) 
for idx in uniqueRecordingData.index:
    subjectName = ('TW_' + str(uniqueRecordingData['SUBJECT_NAME'][idx]))
    sessionId = (uniqueRecordingData['SESSION'][idx])
    recordingId = (uniqueRecordingData['RECORDING'][idx])
    fileId = uniqueRecordingData['FILE'][idx]
    
    vr_key = dict(subject=subjectName, session_id=sessionId, recording_id=recordingId)

    params={'save_as_csv':True}
    model_name = (model.Model & 'model_name LIKE "%' + projectFileName +'%"').fetch1('model_name')
    splitPath = os.path.split(uniqueRecordingData['VIDEOPATH_INBOX'][idx])
    poseOutputDir = splitPath[0] #os.path.join(rootPath,'Outbox',subjectName,str(sessionId))
    
    task_key = {**vr_key, 'model_name': model_name, 'task_mode':'trigger', 'pose_estimation_output_dir':poseOutputDir}
    # task_keyPrimaryOnly = dict([((currKey,task_key[currKey])) for currKey in allPoseEstimationPrimaryKeys])
    

    # if any([currKey == task_keyPrimaryOnly for currKey in allPoseEstimationPrimaryKeyData]):
    #     print(subjectName + '_' + str(sessionId) + '_' + ': primary key values already in use in model.PoseEstimationTask')
    # else:
    model.PoseEstimationTask.insert1(task_key)

    model.PoseEstimation.populate(task_key)

#### 2.5B) Insert pose estimation task (inference task)(s) manually

In [ ]:
# # Manual Step to insert entry into Pose Estimation Task

# # Choose a video recording by manually specifying subject, sesison and recording 
# # Or another method
# # vr_key = dict(subject=subjectName,
# #             session_id=sessionId,
# #             recording_id=recordingId)

# task_mode = "trigger" # Can be set to trigger (to trigger inference) 
#                       # or load (to load external results into database)

# #  Params Optional(default None). Parameters passed to DLC's analyze_videos:
# #                 videotype, gputouse, save_as_csv, batchsize, cropping, TFGPUinference,
# #                 dynamic, robust_nframes, allow_growth, use_shelve
# params={'save_as_csv':True}
# model_name = (model.Model & 'model_name LIKE "%' + projectFileName +'%"').fetch1('model_name')
# task_key = {**recording_key, 'model_name': model_name}
# # Insert PoseEstimationTask
# poseOutputDir = os.path.join(rootPath,'Outbox',subjectName,str(sessionid))
# poseOutputDir = r'D:\Janet_DJ_test\Inbox\TW_102\20231008\device_0_recording_0_model_TW_SideCamera-TW-2023-10-26'
# model.PoseEstimationTask.insert1({
#         **task_key,
#         'task_mode':'trigger',
#         'pose_estimation_output_dir':poseOutputDir
# })

# # Retrieve task key associated with speific model_name
# task_key = (model.PoseEstimationTask & vr_key & {'model': model_name}).fetch1('KEY')
# task_key

# # Run Model Inference Step to populate model.PoseEstimation table
# model.PoseEstimation.populate(task_key)